### urllib3:

- Permite fazer requisições e extrair os dados das páginas.

In [ ]:
import urllib3

# PoolManager: Classe usada para fazer requisições http e pegar seu 
# conteúdo. Pode fazer um pool de conexões, ou seja, conexões com
# várias páginas web.
http = urllib3.PoolManager()

# Requisição para uma pagina web
pagina = http.request('GET', 'http://www.iaexpert.com.br')

# Se o código retornado for 200 a conexão funcionou
# Se o código retornado for 404 a conexão não funcionou
pagina.status

# Pegar todas informações da página (Código html)
pagina.data

### Extração de dados de HTML com Beautifullsoup

- Permite a extração mais organizada dos dados da página.

In [ ]:
from bs4 import BeautifulSoup
import urllib3

# Disabilita warnings de certificado de verificação da requisição 
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
http = urllib3.PoolManager()
pagina = http.request('GET', 'https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o')
pagina.status

# soup = sopa (Representa todos os dados da página)
soup = BeautifulSoup(pagina.data, 'lxml')
# Retorna o(s) titulo(s) da página em formato string
soup.title.string
# Retorna todos os links da página
links = soup.find_all('a')
# Número de links encontrados
len(links)
# Mostra todos os links
for link in links:
    # Retorna o atributo o conteúdo do atributo recebido
    print(link.get('href'))
    # Retorna conteúdo (Título do link)
    print(link.contents)

### Crawler - Busca de documentos 1

- Percorre todos os links da página e adiciona todos os links em uma lista, depois abre todos os links e adiciona os seus links na lista, e repete o processo até uma determinada profundiade.
- Baseado na página inicial, ele abre os links das páginas ligadas a ela.

In [1]:
import urllib3
import re
import nltk
# Biblioteca responsável por liga o python ao banco de dados mysql
import pymysql
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# (use_unicode e charset): Atributos que indicam a formatação a ser utilizada na base de dados.
#   - Isso é importante para palavras retiradas da web, para evitar erros de formatação.

# Função que indexa url recebida como parâmetro e suas palavras, na base dados.
#   - Palavras são inseridas na tabela de palavras.
#   - Url é inserida na tabela de urls.
#   - É realizado a ligação de cada palavra a url, assim como é indicado a localização que a palavra se encontra no documento.
def indexador(url, sopa):
    # Verifica se url já foi indexada.
    indexada = paginaIndexada(url)
    # Se a url já foi indexada a função retorna.
    if indexada == -2:
        return
    # Se a url ainda não foi indexada, ela é inserida na base de dados.
    elif indexada == -1:
        idnova_pagina = insertPagina(url)
    # Se a url já foi indexada, porém ainda não tem palavras cadastradas, seu endereço é armazenado.
    elif indexada > 0:
        idnova_pagina = indexada

    # Recebendo conteúdo principal de cada página como uma grande string.    
    texto = getTexto(sopa)
    # Separando palavras da página a partir de seus radicais e armazenando todos os radicais em uma lista.
    palavras = separaPalavras(texto)

    for i in range(len(palavras)):
        palavra = palavras[i]
        # Verificando se a palavra já está indexada na base de dados.
        # Se não estiver indexada na base de dados, inserir.
        idpalavra = palavraIndexada(palavra)
        if idpalavra == -1:
            idpalavra = insertPalavra(palavra)
        # Unindo palavra a url, na base dados.
        # Informando a localização no qual a palavra se encontra na base de dados.
        # Para essa função, todas as palavras vão ser inseridas.
        insertPalavraLocalizacao(idnova_pagina, idpalavra, i)    


def separaPalavras(texto):
    stop = nltk.corpus.stopwords.words('portuguese')
    stemmer = nltk.stem.RSLPStemmer()
    splitter = re.compile('\\W+')
    lista_palavras = []
    lista = [p for p in splitter.split(texto) if p != '']
    for p in lista:
        if stemmer.stem(p.lower()) not in stop:
            if len(p) > 1:
                lista_palavras.append(stemmer.stem(p.lower()))
    return lista_palavras


# - Função que une uma palavra a uma url, inserindo essa informação na base de dados.
# - Também é inserido a localização no documento dessa palavra na página.
#   - Isso vai ser usado mais na frente para a otimização dos resultados de busca.
def insertPalavraLocalizacao(idurl, idpalavra, localizacao):
    conexao = pymysql.connect(host='localhost', user='root', passwd='123456', db='indice', autocommit = True)
    cursor = conexao.cursor()
    # Inserido "idurl", "idpalavra" e "localizacao" recebidos como parâmetro, na tabela de palavra_localizacao.
    cursor.execute("insert into palavra_localizacao (idurl, idpalavra, localizacao) values (%s, %s, %s)", (idurl, idpalavra, localizacao))
    idpalavra_localizacao = cursor.lastrowid
    cursor.close()
    conexao.close()
    # Retornar o id da palavra_localizacao.
    return idpalavra_localizacao


# - Função que insere a palavra recebida como parâmetro, na tabela de palavras.
def insertPalavra(palavra):
    conexao = pymysql.connect(host='localhost', user='root', passwd='123456', db='indice', autocommit = True, use_unicode=True, charset="utf8mb4")
    cursor = conexao.cursor()
    # - Inserindo palavra recebida, na tabela de palavras da base de dados.
    # - O id de cada palavra é gerado automaticamente conforme a inserção das palavras.
    cursor.execute("insert into palavras (palavra) values (%s)", palavra)
    idpalavra = cursor.lastrowid
    cursor.close()
    conexao.close()
    # Retornar o id no qual a palavra foi inserida.
    return idpalavra


# - Função que verifica se uma palavra já foi indexada na tabela de palavras da base de dados.
# - Seu principal uso é para controlar as palavras que são inseridas na base de dados, de maneira que não seja possível inserir palavras repetidas.
def palavraIndexada(palavra):
    # (retorno == -1) -> palavra não cadastrada.
    retorno = -1
    conexao = pymysql.connect(host='localhost', user='root', passwd='123456', db='indice', use_unicode=True, charset="utf8mb4")
    cursor = conexao.cursor()
    # Realizando consulta para verificar se existe alguma palavra igual a palavra recebida como parâmetro.
    cursor.execute('select idpalavra from palavras where palavra = %s', palavra)
    # Se algum registro foi retornado, quer dizer que a palavra já existe na base de dados.
    if cursor.rowcount > 0:
        # (retorno = cursor.fetchone()[0]) -> palavra cadastrada, então a função retorna o seu idpalavra.
        retorno = cursor.fetchone()[0]
    cursor.close()
    conexao.close()
        
    return retorno


# - Função que insere a url recebida como parâmetro, na tabela de urls.
def insertPagina(url):
    # (autocommit): Permite gravar alterações na base de dados.
    conexao = pymysql.connect(host='localhost', user='root', passwd='123456', db='indice', autocommit = True, use_unicode=True, charset="utf8mb4")
    cursor = conexao.cursor()
    # - Inserindo url recebida, na tabela de urls da base de dados.
    # - O id de cada url é gerado automaticamente conforme a inserção das urls.
    cursor.execute("insert into urls (url) values (%s)", url)
    # lastrowid: retorna o último id que foi inserido na base de dados.
    # - Para execuções concorrentes/paralelas isso é um problema, porém como o objetivo desse crawler é o aprendizado então é válido.
    idpagina = cursor.lastrowid
    cursor.close()
    conexao.close()
    # Retornar o id no qual a url foi inserida.
    return idpagina


# - Função que verifica se a url de uma página já foi indexada na base de dados.
def paginaIndexada(url):
    # (retorno == -1) -> url não cadastrada
    retorno = -1
    # Criando conexão com a base de dados.
    conexao = pymysql.connect(host='localhost', user='root', passwd='123456', db='indice')
    # cursor(): objeto da biblioteca pymsql responsável pela realização das consultas ao banco de dados.
    
    # Criando cursor() para url.
    cursorUrl = conexao.cursor()
    # execute(): método de cursor() que possibilita a execução de consultas a base de dados.
    
    # Realizando consulta para verificar se existe alguma url igual a url passada cadastrada na base de dados.
    cursorUrl.execute('select idurl from urls where url = %s', url)
    # rowcount: retorna o número de linhas da tabela que foi consultada.
    
    # - Se o número de linhas for maior que zero, quer dizer a url recebida já foi cadastrada na base de dados, anteriormente.
    if cursorUrl.rowcount > 0:
        # - Se url já está cadastrada, verificar se já existe alguma palavra cadastrada para essa url.
        
        # Recebendo o idurl da url.
        idurl = cursorUrl.fetchone()[0]
        # Criando cursor() para palavra.
        cursorPalavra = conexao.cursor()
        # Realizando consulta para verificar se existe alguma palavra cadastrada na tabela palavra_localizacao da base de dados.
        cursorPalavra.execute('select idurl from palavra_localizacao where idurl = %s', idurl)
        # - Se o número de linhas for maior que zero, quer dizer que existe palavras cadastrada para a url na base de dados.
        if cursorPalavra.rowcount > 0:
            # (retorno == -2) -> url cadastrada com palavras cadastradas.
            retorno = -2
        else:
            # (retorno == idurl) -> url cadastrada sem palavras cadastradas.
            retorno = idurl
        # Fechamento do cursor. (Liberando memória, pois isso é realizado dinâmicamente).
        cursorPalavra.close()
    # Fechamento do cursor. (Liberando memória, pois isso é realizado dinâmicamente).
    cursorUrl.close()
    # Fechamento da conexão com a base de dados. (Liberando memória, pois isso é realizado dinâmicamente).
    conexao.close()
    
    return retorno


# - Função que realiza o Pré-processamento dos textos a partir da separação das palavras relevantes do texto e do armazenamento 
# de seus radicais.
# - As stopwords são desconsideradas.
# - É permitido o armazenamento de palavras duplicadas.
# - Retorna uma lista de palavras, considerando os parâmetro acima.
def separaPalavras(texto):
    stop = nltk.corpus.stopwords.words('portuguese')
    splitter = re.compile('\\W+')
    stemmer = nltk.stem.RSLPStemmer()

    lista_palavras = []
    lista = [p for p in splitter.split(texto) if p != '']

    for p in lista:
        if p.lower() not in stop:
            if len(p) > 1:
                lista_palavras.append(stemmer.stem(p).lower())
    return lista_palavras


# - Função que realiza o Pré-processamento dos textos a partir de remoção de tags HTML.
# - Retorna uma string gigante que contém o conteúdo relevante de cada página.
def getTexto(soup):
    for tags in soup(['script', 'style']):
        tags.decompose()

    return ' '.join(soup.stripped_strings)


# Importante: O urllib3 precisa da url completa, senão não funciona.

# - Busca links e trata links das páginas recebida.
# - A profundide indica o quão profundo será a busca, como ilustrado no Exemplo 1. 
# - Para uma busca muito profunda é necessário ter um computador potente e uma 
# grande banda de internet.

# Exemplo 1:
# profundidade = 1 -> Será percorrido e gerado os links apenas da lista de páginas recebidas.
# profundidade = 2 -> Será percorrido e gerado os links, tanto da lista de páginas recebidas,
# como da lista de novas páginas geradas a partir dos links gerados a partir da lista de páginas
# recebidas com parâmetro.

# - Aumentar a profundidade significa repetir esse raciocíonio de maneira sucessiva.
# - Se for digitado uma profundidade muito grande será gerado um "looping infinito".

# Como esse é um código de teste, ele vai ser executado e não vai retorna nada.
def crawl(paginas, profundidade):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    for i in range(profundidade):
        # É utilizado um set para evitar links repetidos
        novas_paginas = set()
        # Percorre todas as páginas passadas como parâmetro
        for pagina in paginas:
            http = urllib3.PoolManager()
            try:
                dados_pagina = http.request('GET', pagina)
            except:
                print("Erro ao abria página: " + pagina)
                continue
            
            soup = BeautifulSoup(dados_pagina.data, 'lxml')
            # Indexando página
            indexador(pagina, soup)
            links = soup.find_all('a')
            
            # Percorrer links da página e adicionalos a estrutura de "novas_paginas"
            for link in links:
                # Se for um link válido, ele entra nessa condição
                if 'href' in link.attrs:
                    # - Junta uma url base com uma url relativa
                    # - Torna os links válidos, pois adiciona a parte base do inicio que estava faltando.
                    url = urljoin(pagina, str(link.get('href')))
                    # Retira links inválidos
                    if url.find("'") != -1:
                        continue

                    # Retira links que apontam para própria página
                    url = url.split('#')[0]

                    # Faz mais uma verificação de controle e armazena todos os links de cada página rodada na estrutua "novas_paginas".
                    if url[0:4] == 'http':
                        novas_paginas.add(url)

            # - Faz paginas receber novas páginas, depois que todos links de todas as páginas que foram recebidas como parâmetro foram 
            # armazenados no conjunto "novas páginas".
            # - Isso vale tanto pra primeira execução, quanto para execução em uma profundidade maior.
            paginas = novas_paginas


# listapaginas = ['https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o']
# listapaginas = ['https://dhauz.com/']
# crawl(listapaginas, 1)

# Próximos passos:

# 1 - Deletar base de dados (OK)
# 2 - Terminar de assistir aulas do módulo (OK)
# 3 - Organizar anotações e simplificar todas anotações do módulo. (Revisão). (OK)
# 4 - Começar próximo módulo e seguir metodologia semelhante.

# Sugestões:

# 1- Quando terminar o curso, otimizar pontos de melhoria do código. (Sempre testando para ver se está tudo ok).

### Pré-processamento dos textos - Remoção de tags HTML

- Redução do conteúdo desnecessário que vem junto com o texto extraido na requsição.
- Remoção das tags de 'style' e 'script'.
- Pegar conteúdo das tags que sobraram e forma uma string gigante.

In [ ]:
import urllib3
from bs4 import BeautifulSoup

http = urllib3.PoolManager()
pagina = http.request('GET', 'https://pt.wikipedia.org/wiki/Linguagem_de_programa%C3%A7%C3%A3o')

soup = BeautifulSoup(pagina.data, 'lxml')

# Remove tags de script e style
for tags in soup(['script', 'style']):
    tags.decompose()

# Junta todas as strings de conteúdo em uma só, e remove os espaços entre elas
conteudo = ' '.join(soup.stripped_strings)

# print(conteudo)

### Pré-processamento dos textos - Separação das Palavras

- Recebendo uma string grande e quebrando em palavras, descosiderando as stops words.
- Armazenar palavras únicas.

In [ ]:
# Importando expressões regulares. (Aplicar filtros em strings).
import re 
# Importando biblioteca específica para processamento de linguagem natural.
# -> Importante para identificação das stops words.
# -> Stops words: Para que não tem significados sozinhas, portanto não são relevantes para identidade da página.
import nltk 

# Fazer downloads do pacote da biblioteca 
# nltk.download('stopwords')

# - Forma uma lista de stop words de acordo com o idioma.
# - Todas letras são em minúsculo, então é importante definir um padrão de letras minúsculas na verificação.
stop = nltk.corpus.stopwords.words('portuguese')

# - 1) Separação das palavras e Remoção das stops words:

# W -> buscar qualquer caracter que não é um palavra.
# + -> pode ter qualquer elemento .
splitter = re.compile('\\W+')

lista_palavras = []
# - Quebra as palavras do texto dentro de split(), de maneira que quando encontra algum texto que não é um caracter
# ele quebra a string passada como parâmetro e armazena na lista.
# - Ignora os espaços vazios.

# - Não funciona em todos os casos.
# - Para funcionar para todos os casos é necessário adicionar mais filtros para tornar a indetificação de palavras válidas
# mais eficiente.
lista = [p for p in splitter.split('Este lugar lugar é apavorante a b c++') if p != '']

for p in lista:
    # Não adiciona as stops words ou letras sozinhas.
    if p.lower() not in stop:
        if len(p) > 1:
            lista_palavras.append(p.lower())

print(lista_palavras)

### Pré-processamento dos textos - Extração do radical

- Extrair o radical de cada palavra que não é uma stopword.
- Importante para diminuir o número de palavras que serão armazenadas na base de dados, pois como palavras que possuem o mesmo racial possuem o mesmo sentido é possível fazer essa simplificação.
- Segundo a maneira que foi implementado é possível adicionar palavras repetidas.

In [ ]:
import re 
import nltk

stop = nltk.corpus.stopwords.words('portuguese')
# nltk.download('rslp')

splitter = re.compile('\\W+')
# RSLPStemmer: Classe usada para extrair o radical de cada palavra.
# - Muito util para diminuir o número de palavras únicas a serem armazenadas no banco de dados
# - Exemplo de extração de radical: "nova" e "novamente" podem ser armazenados apenas como uma palavra única, apartir de seu
# radical "nov".

stemmer = nltk.stem.RSLPStemmer()

lista_palavras = []
lista = [p for p in splitter.split('Este lugar lugar é apavorante a b c++') if p != '']

for p in lista:
    if p.lower() not in stop:
        if len(p) > 1:
            # Para uma codificação correta a alteração deve ser feita apenas na hora de armazenar a palavra na lista, para
            # não ocorrer conflito na verificação das stopswords.
            # Lembrando que: Nesse caso há a possibilidade de ser inseridos valores repetidos, então em vez de uma lista,
            # pode ser mais interessante o uso de um set().
            lista_palavras.append(stemmer.stem(p).lower())

# stemmer.stem('nova')
# stemmer.stem('novamente')

print(lista_palavras)